## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SBBBBBBS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1008
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+--------------------+--------------------+--------------------+------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|               res_1|               res_2|               res_3|          res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+------------------+------------------+---------+--------+--------+---------+-----------+--------------------+--------------------+--------------------+------------------+-----------+-----------+------+--------------------+------+
|  5236.56372535678|-524.2927399414914|28.596229640470586|    454.2|SBBBBBBS| 47.2256| -97.1919|       SNWD| 0.02943165541547636|0.019702368195064154|0.019673424705629848|0.56

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [16]:
#extract longitude and latitude for each station
feature='coeff_'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


station    year      coeff_3
0  USC00214106  1969.0    28.596230
1  USC00325220  1969.0    77.356006
2  USC00216276  1969.0 -1845.489009
3  USW00094967  1997.0   101.899035
4  USW00094967  1969.0  -961.312053

In [17]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.tail(10)

station  US1MNCY0001  USC00210018  USC00211891  USC00212142  USC00212916  \
year                                                                       
2001.0           NaN          NaN   -15.573328          NaN          NaN   
2003.0           NaN          NaN          NaN          NaN          NaN   
2004.0           NaN          NaN  -224.721605          NaN          NaN   
2005.0           NaN          NaN  -555.618023          NaN          NaN   
2006.0           NaN          NaN   715.499306          NaN          NaN   
2007.0           NaN          NaN   594.817517          NaN          NaN   
2008.0           NaN          NaN   250.805141          NaN          NaN   
2009.0           NaN          NaN          NaN          NaN          NaN   
2010.0   -436.167735          NaN  -498.187219          NaN          NaN   
2011.0           NaN          NaN          NaN          NaN          NaN   

station  USC00213104  USC00213206  USC00213463  USC00213587  USC00214106  \
year                                                                       
2001.0           NaN          NaN          NaN          NaN   325.286064   
2003.0           NaN          NaN          NaN          NaN   304.820388   
2004.0           NaN          NaN          NaN          NaN   -27.405879   
2005.0           NaN          NaN          NaN          NaN  -190.266889   
2006.0           NaN          NaN          NaN          NaN   317.160756   
2007.0           NaN          NaN          NaN          NaN   868.020133   
2008.0           NaN          NaN          NaN          NaN   250.510279   
2009.0           NaN          NaN          NaN          NaN          NaN   
2010.0           NaN          NaN          NaN          NaN  -238.364182   
2011.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00325730  USC00325764  USC00326105  USC00326565  \
year        ...                                                            
2001.0      ...        256.262662          NaN     3.362520          NaN   
2003.0      ...         45.654310          NaN          NaN          NaN   
2004.0      ...       -327.009286          NaN  -135.455402          NaN   
2005.0      ...               NaN          NaN          NaN          NaN   
2006.0      ...               NaN          NaN          NaN          NaN   
2007.0      ...        621.655003          NaN   661.090689          NaN   
2008.0      ...        239.496464          NaN   188.709214          NaN   
2009.0      ...       -551.123333          NaN    17.354068          NaN   
2010.0      ...               NaN          NaN  -240.480178          NaN   
2011.0      ...               NaN          NaN          NaN          NaN   

station  USC00327986  USC00328937  USC00329185  USW00014914  USW00014919  \
year                                                                       
2001.0           NaN          NaN          NaN          NaN          NaN   
2003.0           NaN          NaN          NaN          NaN          NaN   
2004.0           NaN          NaN          NaN          NaN          NaN   
2005.0           NaN          NaN          NaN          NaN          NaN   
2006.0           NaN          NaN          NaN          NaN          NaN   
2007.0           NaN          NaN          NaN          NaN          NaN   
2008.0           NaN          NaN          NaN          NaN          NaN   
2009.0           NaN          NaN          NaN          NaN          NaN   
2010.0           NaN          NaN          NaN          NaN          NaN   
2011.0           NaN          NaN          NaN          NaN          NaN   

station  USW00094967  
year                  
2001.0           NaN  
2003.0           NaN  
2004.0           NaN  
2005.0           NaN  
2006.0           NaN  
2007.0           NaN  
2008.0           NaN  
2009.0           NaN  
2010.0           NaN  
2011.0           NaN  

[10 rows x 47 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [18]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print mean_by_year.shape
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

(98,)
total RMS                   =  588.823156235
RMS removing mean-by-station=  566.242330464
RMS removing mean-by-year   =  408.004173428


In [10]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 631.957294164
0 after removing mean by year    = 329.960575426
0 after removing mean by stations= 310.920358488
1 after removing mean by year    = 309.659157759
1 after removing mean by stations= 309.523374797
2 after removing mean by year    = 309.504389197
2 after removing mean by stations= 309.499416665
3 after removing mean by year    = 309.496335626
3 after removing mean by stations= 309.493603558
4 after removing mean by year    = 309.491011026
4 after removing mean by stations= 309.488525441
